In [6]:
import collections
import urllib.request
import json
import requests
import csv
import os
import shutil
import collections
import pandas as pd

In [7]:
url_tissue_rui = "https://ccf-api.hubmapconsortium.org/v1/hubmap/rui_locations.jsonld"
url_reference_organ_data = "https://hubmapconsortium.github.io/ccf-ui/rui/assets/reference-organ-data.json"
url_collision_detection = "http://192.168.1.100:8080/get-collisions"

Step 1: save all the hubmap, CxG,GTex.. datasets in one folder

In [3]:
def copy_csv_files_in_one_folder(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder in os.listdir(input_dir):
        full_folder = os.path.join(input_dir, folder)
        if os.path.isdir(full_folder):
            for file_name in os.listdir(full_folder):
                if file_name.endswith('.csv'):
                    abs_file_path = os.path.join(full_folder, file_name)
                    output_file_path = os.path.join(output_dir, file_name)

                    if folder.find('CxG') >= 0:
                        index = folder.find('CxG')
                        organ = folder[:index-1]
                        new_file_name = file_name[:-4] + '_' + organ + '.csv'
                        output_file_path = os.path.join(output_dir, new_file_name)

                    shutil.copy(abs_file_path, output_file_path)

In [22]:
input_dir = 'C:/Users/catherine/Desktop/research/hubmap/tissue-bar-graphs/csv'
output_dir = '../all_csv_2'
copy_csv_files_in_one_folder(input_dir, output_dir)

Step 2: Compare Training/Prediction Table with CT counts in bar-graph-portal and rui locations in all_rui_location_by_dataset_id.json

In [8]:
def load_bar_graph_portal_dataset(csv_dir_path):
    dir_list = os.listdir(csv_dir_path)
    return [e[:-4] for e in dir_list if e.endswith('.csv')]

def load_all_rui_location_by_dataset_id(json_path):
    dic = {}
    datasets_with_rui_id = {}
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    for dataset_id, values in json_data.items():

        rui_location = None
        if 'rui-location' in values:
            rui_location = values['rui-location']
        elif 'rui_location' in values:
            rui_location = values['rui_location']

        if not rui_location:
            continue

        # hubmap dataset, use hubmap-id as tissue block id
        if 'hubmap-id' in values:
            unified_tissue_block_id = values['hubmap-id']
        # gtex dataset, use sample-id as tissue block id
        elif dataset_id.startswith('GTEX'):
            unified_tissue_block_id = values['sample-id']
        # CxG dataset, use dataset_id as tissue block id, in all_rui_location_by_dataset_id, dataset_id for CxG is dataset_id_donor_organ_(AS).
        else:
            unified_tissue_block_id = dataset_id

        datasets_with_rui_id[dataset_id] = unified_tissue_block_id
        dic[unified_tissue_block_id] = rui_location

    return dic, datasets_with_rui_id

Step 3(extra): combine dataset_id - uuid - h5ad(exist)

In [5]:
descendant_path = "../Vicky/finddesecendants.csv"
descendant_h5ad_path = "../Vicky/findh5addesecendants.csv"
URL = f"https://entity.api.hubmapconsortium.org/entities/"
headers = {
    'Authorization': 'Bearer ' + 'AgOdjmzyX2q4NEaOEy8P8n10dXXlDB89y0J6DQ4EOYnGOKoDP4TnCgP8r2e9rNkrrMalmYPbPm9W49F8e8gPJSWvnl'
}

In [6]:
def get_hubmap_id(uuid):
    print(URL + uuid)
    resp = requests.get(URL + uuid, headers=headers).json()
    if 'hubmap_id' in resp:
        return resp['hubmap_id']

In [22]:
descendant = pd.read_csv(descendant_path)
descendant_h5ad = pd.read_csv(descendant_h5ad_path)

descendant = descendant[['hubmap_id', 'uuid']].drop_duplicates()
descendant_h5ad = descendant_h5ad[['uuid', 'descendants', 'has_h5']]
descendant_h5ad_exist = descendant_h5ad[descendant_h5ad['has_h5'] == True]
dataset_id_with_h5ad = descendant.merge(descendant_h5ad_exist, on='uuid', how='left')
dataset_id_with_h5ad = dataset_id_with_h5ad[dataset_id_with_h5ad['has_h5'] == True]
dataset_set = set(dataset_id_with_h5ad['hubmap_id'])
print(len(dataset_set))

264


In [27]:
dataset_id_with_h5ad['exact_hubmap_id'] = dataset_id_with_h5ad['descendants'].apply(get_hubmap_id)

https://entity.api.hubmapconsortium.org/entities/0d1eb3d774a694b79e844987f771b183
https://entity.api.hubmapconsortium.org/entities/107f237dc1c866e9e9b215c7354c8d6a
https://entity.api.hubmapconsortium.org/entities/0d1eb3d774a694b79e844987f771b183
https://entity.api.hubmapconsortium.org/entities/107f237dc1c866e9e9b215c7354c8d6a
https://entity.api.hubmapconsortium.org/entities/82400009b02f7651f3edf5b292f7cd9a
https://entity.api.hubmapconsortium.org/entities/82400009b02f7651f3edf5b292f7cd9a
https://entity.api.hubmapconsortium.org/entities/704d16116f29994ab0e714f4afdb5516
https://entity.api.hubmapconsortium.org/entities/704d16116f29994ab0e714f4afdb5516
https://entity.api.hubmapconsortium.org/entities/dd3e6da115fd13eee62d9a4fba2324ae
https://entity.api.hubmapconsortium.org/entities/dd3e6da115fd13eee62d9a4fba2324ae
https://entity.api.hubmapconsortium.org/entities/7479e6dbfa3b3a8e1e2d4f78cbe0f9e4
https://entity.api.hubmapconsortium.org/entities/7479e6dbfa3b3a8e1e2d4f78cbe0f9e4
https://entity.a

In [29]:
dataset_id_with_h5ad.to_csv('../Vicky/dataset_with_exact_hubmap_id.csv', index=False)

Step 3: check Training/Prediction Table

In [9]:
csv_dir_path = '../all_csv_2'
json_path = 'all_rui_location_by_dataset_id_new.json'
training_prediction_path = './CTPop - Training_Prediction Datasets - Training_Prediction.csv'
dataset_id_with_exact_hubmap_id_path = '../Vicky/dataset_with_exact_hubmap_id.csv'

In [10]:
bar_graph_portal_dataset = load_bar_graph_portal_dataset(csv_dir_path)
tissue_block_id_rui, datasets_with_rui_id = load_all_rui_location_by_dataset_id(json_path)

training_prediction = pd.read_csv(training_prediction_path)
training_prediction = training_prediction[~(training_prediction['excluded'] == True)]

dataset_id_with_h5ad = pd.read_csv(dataset_id_with_exact_hubmap_id_path)

In [11]:
hubmap_id_in_portal = {e: True for e in bar_graph_portal_dataset}
dataset_id_with_h5ad['exist_in_portal'] = dataset_id_with_h5ad['exact_hubmap_id'].map(hubmap_id_in_portal)
dataset_id_exist_in_portal = dataset_id_with_h5ad[dataset_id_with_h5ad['exist_in_portal'] == True]
mapping = dataset_id_exist_in_portal[['hubmap_id', 'exact_hubmap_id']].drop_duplicates()
mapping_dataset_id_to_exact_hubmap_id = {row['hubmap_id']: row['exact_hubmap_id'] for index, row in mapping.iterrows()}
# special case: HBM945.QNRF.244 has two descedants
mapping_dataset_id_to_exact_hubmap_id['HBM945.QNRF.244'] = 'HBM656.TJNG.945'

In [13]:
for index, row in training_prediction.iterrows():
    source, dataset_id, hubmap_id, sample_id, CxG_dataset_id_donor_id_organ = row['source'], row['dataset_id'], row['HuBMAP_tissue_block_id'], row['sample_id'], row['CxG_dataset_id_donor_id_organ']

    unique_tissue_block_id = "#############"
    if source == 'HuBMAP':
        unique_tissue_block_id = hubmap_id
    elif source == 'GTEx':
        unique_tissue_block_id = sample_id
    elif source == 'CxG':
        unique_tissue_block_id = CxG_dataset_id_donor_id_organ
        dataset_id = CxG_dataset_id_donor_id_organ

    if dataset_id not in bar_graph_portal_dataset:
        # print('row: ', index + 2, ' dataset_id: ', dataset_id, ' not in bar graph portal')

        if dataset_id not in mapping_dataset_id_to_exact_hubmap_id:
            print('row: ', index + 2, ' dataset_id: ', dataset_id, ' not in descendant table')
            pass
        pass


    if source == 'CxG':
        if CxG_dataset_id_donor_id_organ not in datasets_with_rui_id:
            print('CxG dataset id ', CxG_dataset_id_donor_id_organ, ' does not exist')

    if dataset_id not in datasets_with_rui_id:
        print('row: ', index + 2, 'dataset_id: ', dataset_id, ' unique tissue block id: ', unique_tissue_block_id, ' not in all_rui_location_by_dataset_id.json')
        continue
    if source == 'HuBMAP':
        if datasets_with_rui_id[dataset_id] != hubmap_id:
            print('unmatched hubmap id for dataset id: ', dataset_id, " hubmap id in big table: ", hubmap_id, " hubmap id in json: ", datasets_with_rui_id[dataset_id])
    if source == 'GTEx':
        if datasets_with_rui_id[dataset_id] != sample_id:
            print('unmatched sample id for dataset id: ', dataset_id, " sample id in big table: ", sample_id, " sample id in json: ", datasets_with_rui_id[dataset_id])



row:  289 dataset_id:  HBM378.WGXD.394  unique tissue block id:  HBM999.WVWP.899  not in all_rui_location_by_dataset_id.json
row:  674 dataset_id:  HBM827.NQQL.854  unique tissue block id:  HBM776.MWMG.262  not in all_rui_location_by_dataset_id.json
row:  716 dataset_id:  HBM854.LQKL.226  unique tissue block id:  HBM938.JJMV.536  not in all_rui_location_by_dataset_id.json


In [14]:
def get_unified_tissue_block_id(row):
    source, dataset_id, hubmap_id, sample_id, CxG_dataset_id_donor_id_organ = row['source'], row['dataset_id'], row['HuBMAP_tissue_block_id'], row['sample_id'], row['CxG_dataset_id_donor_id_organ']

    unified_tissue_block_id = None
    if source == 'HuBMAP':
        unified_tissue_block_id = hubmap_id
    if source == 'GTEx':
        unified_tissue_block_id = sample_id
    if source == 'CxG':
        unified_tissue_block_id = CxG_dataset_id_donor_id_organ

    return unified_tissue_block_id

def get_csv_file_name(row):
    source, dataset_id, hubmap_id, sample_id, CxG_dataset_id_donor_id_organ = row['source'], row['dataset_id'], row['HuBMAP_tissue_block_id'], row['sample_id'], row['CxG_dataset_id_donor_id_organ']

    file_name = None
    if source == 'HuBMAP':
        if dataset_id in mapping_dataset_id_to_exact_hubmap_id:
            dataset_id = mapping_dataset_id_to_exact_hubmap_id[dataset_id]
        file_name = dataset_id + '.csv'
    if source == 'GTEx':
        file_name = dataset_id + '.csv'
    if source == 'CxG':
        file_name = CxG_dataset_id_donor_id_organ + '.csv'

    return file_name

In [15]:
training_prediction['united_tissue_block_id'] = training_prediction.apply(get_unified_tissue_block_id, axis=1)
training_prediction['csv_file_path'] = training_prediction.apply(get_csv_file_name, axis=1)
training_prediction['rui_location'] = training_prediction['united_tissue_block_id'].map(tissue_block_id_rui)

In [16]:
training_prediction

,dataset_id,source,alt_source,is_azimuth_reference,excluded,reason_for_exclusion,has_h5ad_file,paper_id,HuBMAP_tissue_block_id,HuBMAP_donor_id,...,non_hubmap_donor_id,HuBMAP_publication_status,CxG_dataset_id_donor_id_organ,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,united_tissue_block_id,csv_file_path,rui_location
3,GTEX-1HSMQ-5007,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://gtexportal.org/home/tissue/Skin_Sun_Ex...,GTEX-1HSMQ-5007.csv,{'@context': 'https://hubmapconsortium.github....
4,GTEX-15EOM-5003,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://gtexportal.org/home/tissue/Skin_Sun_Ex...,GTEX-15EOM-5003.csv,{'@context': 'https://hubmapconsortium.github....
5,GTEX-1CAMS-5015,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://gtexportal.org/home/tissue/Breast#FTis...,GTEX-1CAMS-5015.csv,{'@context': 'https://hubmapconsortium.github....
6,GTEX-1MCC2-5013,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://gtexportal.org/home/tissue/Breast#FTis...,GTEX-1MCC2-5013.csv,{'@context': 'https://hubmapconsortium.github....
7,GTEX-1R9PN-5002,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://gtexportal.org/home/tissue/Breast#FTis...,GTEX-1R9PN-5002.csv,{'@context': 'https://hubmapconsortium.github....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,b4c3c03b-99cd-407a-a824-1875e00a04ad,CxG,Teichmann,True,NaN,NaN,NaN,https://doi.org/10.1038/s41586-020-2797-4,NaN,NaN,...,H7,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,NaN,NaN,NaN,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,{'@context': 'https://hubmapconsortium.github....
4156,b4c3c03b-99cd-407a-a824-1875e00a04ad,CxG,Teichmann,True,NaN,NaN,NaN,https://doi.org/10.1038/s41586-020-2797-4,NaN,NaN,...,H7,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,NaN,NaN,NaN,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,{'@context': 'https://hubmapconsortium.github....
4157,b4c3c03b-99cd-407a-a824-1875e00a04ad,CxG,Teichmann,True,NaN,NaN,NaN,https://doi.org/10.1038/s41586-020-2797-4,NaN,NaN,...,H7,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,NaN,NaN,NaN,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,{'@context': 'https://hubmapconsortium.github....
4158,b4c3c03b-99cd-407a-a824-1875e00a04ad,CxG,Teichmann,True,NaN,NaN,NaN,https://doi.org/10.1038/s41586-020-2797-4,NaN,NaN,...,H7,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,NaN,NaN,NaN,NaN,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_...,{'@context': 'https://hubmapconsortium.github....


In [48]:
dic_tissue_block_cell_type = collections.defaultdict(list)
for index, row in training_prediction.iterrows():
    # source, dataset_id, hubmap_id, sample_id, CxG_dataset_id_donor_id_organ = row['source'], row['dataset_id'], row['HuBMAP_tissue_block_id'], row['sample_id'], row['CxG_dataset_id_donor_id_organ']
    united_tissue_block_id, csv_file_path = row['united_tissue_block_id'], row['csv_file_path']
    dic_tissue_block_cell_type[united_tissue_block_id].append(csv_file_path)

cell_summary_AS = collections.defaultdict(list)
collide_with = {}

for united_tissue_block_id in dic_tissue_block_cell_type:
    datasets = dic_tissue_block_cell_type[united_tissue_block_id]
    rui_location = None
    if united_tissue_block_id in tissue_block_id_rui:
        rui_location = tissue_block_id_rui[united_tissue_block_id]
        # the tissue block is not associated with a rui location
        if not rui_location:
            continue

        # 1. compute the average cell type counts for each tissue block
        df_ct = []
        for dataset in datasets:
            df = pd.read_csv(os.path.join(csv_dir_path, dataset))
            df_ct.append(df)

        df_total = pd.concat(df_ct, ignore_index=True)
        df_summary = df_total.groupby('cell_type')['count'].mean()

        # 2. send request to get collision results
        post_response = requests.post(url='https://pfn8zf2gtu.us-east-2.awsapprunner.com/get-collisions', json=rui_location).json()

        for AS_item in post_response:
            # percentage_of_AS = AS_item['percentage_of_AS']
            # percentage_of_tissue_block = AS_item['percentage_of_tissue_block']
            organ = AS_item['organ']
            node_name = AS_item['node_name']
            AS_volume = AS_item['AS_volume']

            if (organ, node_name) not in collide_with:
                # [AS volume, not only collide with, only collide with, number of tissue block with no CT info]
                collide_with[(organ, node_name)] = [AS_volume, 0, 0, 0]

            collide_with[(organ, node_name)][1] += 1

        # 3. estimate the CT population in a certain AS using the tissue blocks that only collide with it
        if len(post_response) == 1:
            AS_item = post_response[0]
            percentage_of_AS = AS_item['percentage_of_AS']
            percentage_of_tissue_block = AS_item['percentage_of_tissue_block']
            organ = AS_item['organ']
            node_name = AS_item['node_name']
            collide_with[(organ, node_name)][2] += 1
            df_summary = df_summary * percentage_of_tissue_block / percentage_of_AS
            cell_summary_AS[(organ, node_name)].append(df_summary)


# process cell summaries of AS
def process_cell_summaries(cell_summary_AS):
    avg_cell_summary_AS = {}

    for organ_AS, df_summary_list in cell_summary_AS.items():
        df = pd.concat([df.reset_index() for df in df_summary_list], ignore_index=True)
        df = df.groupby('cell_type').sum().reset_index()
        df['count'] = df['count'] / len(df_summary_list)
        avg_cell_summary_AS[organ_AS] = df

    return avg_cell_summary_AS

avg_cell_summary_AS = process_cell_summaries(cell_summary_AS)

In [57]:
cell_type_total = []
for (organ, AS), avg_df_summary in avg_cell_summary_AS.items():
    avg_df_summary['organ'] = organ
    avg_df_summary['AS'] = AS
    cell_type_total.append(avg_df_summary)

cell_type_total = pd.concat(cell_type_total, ignore_index=True)
cell_type_total = cell_type_total[['organ', 'AS', 'cell_type', 'count']]
cell_type_total = cell_type_total.sort_values(by=['organ', 'AS', 'cell_type', 'count'])
cell_type_total.to_csv('./csv/specification_total.csv', index=False)

In [59]:
dict_collide_with = collections.defaultdict(list)

for (organ, AS), info in collide_with.items():
    dict_collide_with['Name_of_organ'].append(organ)
    dict_collide_with['Name_of_colliding_AS'].append(AS)
    dict_collide_with['Volume_of_colliding_AS'].append(info[0])
    dict_collide_with['Number_tissue_blocks_colliding_only_with_this_AS'].append(info[2])
    dict_collide_with['Total_number_colliding_tissue_blocks'].append(info[1])
    dict_collide_with['Number_of_colliding_tissue_blocks with CT info'].append(info[1])
    dict_collide_with['Number_of_colliding_tissue_blocks with no CT info'].append(0)

df_collide_with = pd.DataFrame(dict_collide_with)
df_collide_with.to_csv('./csv/collide_with_0527.csv', index=False)


Step 4: construct all_rui_location_by_dataset_id.json for datasets in Training/Prediction Table

In [42]:
with open(json_path, 'r') as f:
    json_data = json.load(f)
    for index, row in training_prediction.iterrows():
        source, dataset_id, hubmap_id, sample_id, CxG_dataset_id_donor_id_organ = row['source'], row['dataset_id'], row['HuBMAP_tissue_block_id'], row['sample_id'], row['CxG_dataset_id_donor_id_organ']

        if dataset_id not in datasets_with_rui_id:
            if sample_id in tissue_block_id_rui:
                json_data[CxG_dataset_id_donor_id_organ] = {"sample-id": sample_id, "rui-location": tissue_block_id_rui[sample_id]}

    with open("all_rui_location_by_dataset_id_new_test.json", "w") as outfile:
        json.dump(json_data, outfile, indent=4)


In [53]:
training_prediction[training_prediction['source'] == 'GTEx'].groupby('sample_id')['dataset_id'].agg('count')

sample_id
https://gtexportal.org/home/tissue/Breast#FTissueBlocks                        3
https://gtexportal.org/home/tissue/Heart_Left_Ventricle#FTissueBlocks          2
https://gtexportal.org/home/tissue/Heart_Left_Ventricle#MTissueBlocks          1
https://gtexportal.org/home/tissue/Lung#FTissueBlocks                          1
https://gtexportal.org/home/tissue/Lung#MTissueBlocks                          2
https://gtexportal.org/home/tissue/Prostate#MTissueBlocks                      4
https://gtexportal.org/home/tissue/Skin_Sun_Exposed_Lower_leg#FTissueBlocks    1
https://gtexportal.org/home/tissue/Skin_Sun_Exposed_Lower_leg#MTissueBlocks    1
Name: dataset_id, dtype: int64

In [54]:
training_prediction[training_prediction['source'] == 'CxG'].groupby('CxG_dataset_id_donor_id_organ')['dataset_id'].agg('count')

CxG_dataset_id_donor_id_organ
1ca90a2d-2943-483d-b678-b809bf464c30_H18.30.001_Brain                    1
1ca90a2d-2943-483d-b678-b809bf464c30_H18.30.002_Brain                    1
1ca90a2d-2943-483d-b678-b809bf464c30_H19.30.001_Brain                    1
1ca90a2d-2943-483d-b678-b809bf464c30_H19.30.002_Brain                    1
1ca90a2d-2943-483d-b678-b809bf464c30_H19.33.004_Brain                    1
                                                                        ..
b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_heart_left_ventricle       1
b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_heart_right_ventricle      1
b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_interventricular_septum    1
b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_left_cardiac_atrium        1
b4c3c03b-99cd-407a-a824-1875e00a04ad_H7_Heart_right_cardiac_atrium       1
Name: dataset_id, Length: 138, dtype: int64

In [89]:
training_prediction[training_prediction['source'] =='HuBMAP'].groupby('HuBMAP_tissue_block_id')['dataset_id'].agg('count')

HuBMAP_tissue_block_id
HBM223.KDVZ.277    6
HBM224.HCFB.249    2
HBM229.HKHH.537    1
HBM238.QPLB.494    1
HBM242.XXTB.333    1
                  ..
HBM974.MTJZ.548    2
HBM979.SSNH.934    2
HBM996.XSFS.465    1
HBM998.VHPF.947    1
HBM999.WVWP.899    1
Name: dataset_id, Length: 113, dtype: int64

Step 3: for every tissue block, send a request and return the intersected volume and volume of AS.

In [4]:
file_mapping_rui_uuid_hubmap_id = './csv/mapping_rui_uuid_hubmap_id.csv'
file_mapping_rui_hubmap_id = './csv/mapping_rui_hubmap_id.csv'
hubmap_csv_folder = output_dir

In [8]:
mapping_rui_dataset_id = collections.defaultdict(list)

with open(file_mapping_rui_uuid_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        hubmap_id, uuid, rui = row
        mapping_rui_dataset_id[rui].append(hubmap_id)

with open(file_mapping_rui_hubmap_id, 'w', encoding='UTF8', newline="") as f:
    csv_writer = csv.writer(f)
    for rui, hubmap_ids in mapping_rui_dataset_id.items():
        csv_writer.writerow([rui] + hubmap_ids)

In [46]:
mapping_AS_cell_type_count = collections.defaultdict(dict)

with open(file_mapping_rui_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    i = 0
    for row in csv_reader:
        if i == 0:
            i = 1
            continue

        rui_url = row[0]
        hubmap_ids = row[1:]

        dic_cell_type_count = collections.defaultdict(list)
        dic_cell_type_avg_count = collections.Counter()
        for hubmap_id in hubmap_ids:
            csv_path = os.path.join(hubmap_csv_folder, hubmap_id + '.csv')

            with open(csv_path, 'r', encoding='UTF8') as csv_file:
                dataset_csv_reader = csv.reader(csv_file)
                header = next(dataset_csv_reader)
                cell_type_index = header.index('cell_type')
                count_index = header.index('count')

                for item_cell in dataset_csv_reader:
                    dic_cell_type_count[item_cell[cell_type_index]].append(float(item_cell[count_index]))

        for cell, count_list in dic_cell_type_count.items():
            dic_cell_type_avg_count[cell] = sum(count_list) / len(hubmap_ids)

        get_response = requests.get(rui_url).json()
        rui_location = get_response['rui_location']
        post_response = requests.post(url=url_collision_detection, json=rui_location)
        target = rui_location['placement']['target']
        try:
            post_response = post_response.json()

            for AS_item in post_response:
                percentage_of_AS = AS_item['percentage_of_AS']
                percentage_of_tissue_block = AS_item['percentage_of_tissue_block']
                organ = AS_item['organ']
                node_name = AS_item['node_name']

                # if 'Skin' in organ:
                #     print(organ, hubmap_ids)

                for cell, avg_count in dic_cell_type_avg_count.items():
                    # initialize
                    if cell not in mapping_AS_cell_type_count[(organ, node_name)]:
                        mapping_AS_cell_type_count[(organ, node_name)][cell] = []

                    if 'Skin' in organ:
                        mapping_AS_cell_type_count[(organ, node_name)][cell].append(avg_count * percentage_of_tissue_block/ percentage_of_AS)
                    else:
                        mapping_AS_cell_type_count[(organ, node_name)][cell].append(avg_count / percentage_of_AS)
        except:
            pass

result = collections.defaultdict(dict)
for (organ, node_name) in mapping_AS_cell_type_count:
    for cell in mapping_AS_cell_type_count[(organ, node_name)]:
        cell_list = mapping_AS_cell_type_count[(organ, node_name)][cell]
        if 'Skin' in organ:
            print(organ, cell, cell_list)
        if len(cell_list) == 0:
            print(organ, node_name, cell)
        else:
            result[(organ, node_name)][cell] = int(sum(cell_list) / len(cell_list))

# print(mapping_AS_cell_type_count)
# print(result)

VH_M_Kidney_R VH_M_renal_column_R Afferent / Efferent Arteriole Endothelial
VH_M_Kidney_R VH_M_renal_column_R Ascending Thin Limb
VH_F_Skin CD68 
/ Macrophage [88466.59516537667, 70327.46847917805, 74976.07164909289]
VH_F_Skin T-Killer [24034.277299264184, 8625.066888955798, 46366.7811514127]
VH_F_Skin T-reg [80284.71353158461, 56394.6681200956, 62151.21728806384]
VH_F_Skin T-helper [669380.1911646131, 479686.41236269556, 1043745.8395360563]
VH_M_Skin CD68 
/ Macrophage [19888.80507396163, 229856.039523177, 62791.798876364584, 91007.33447535438, 297214.514681459]
VH_M_Skin T-Killer [6894.785758973365, 161355.89529441565, 26013.745248779614, 3914.293955929221, 58605.67895127361]
VH_M_Skin T-reg [2651.8406765282175, 537345.5758389502, 98672.82680571577, 58714.40933893831, 221864.3560298215]
VH_M_Skin T-helper [256963.3615555843, 2715650.1623135614, 740943.2267411021, 353265.02952261217, 2501206.655241856]
VH_F_Kidney_R VH_F_renal_column_R Afferent / Efferent Arteriole Endothelial


In [47]:
result

defaultdict(dict,
            {('VH_M_Kidney_R',
              'VH_M_outer_cortex_of_kidney_R'): {'Afferent / Efferent Arteriole Endothelial': 71974, 'Ascending Thin Limb': 5226589, 'Ascending Vasa Recta Endothelial': 421542, 'B': 15744, 'Classical Dendritic': 488119, 'Connecting Tubule': 3022323, 'Connecting Tubule Intercalated Type A': 27481, 'Connecting Tubule Principal': 664661, 'Cortical Collecting Duct Intercalated Type A': 1154457, 'Cortical Collecting Duct Principal': 1037744, 'Cortical Thick Ascending Limb': 10098949, 'Cortical Vascular Smooth Muscle / Pericyte': 165173, 'Descending Thin Limb Type 1': 1502880, 'Descending Thin Limb Type 2': 2815519, 'Descending Thin Limb Type 3': 118921, 'Descending Vasa Recta Endothelial ': 152455, 'Distal Convoluted Tubule Type 1': 5864833, 'Distal Convoluted Tubule Type 2': 293828, 'Fibroblast': 2058312, 'Glomerular Capillary Endothelial': 492863, 'Inner Medullary Collecting Duct': 4769141, 'Intercalated Type B': 730093, 'Lymphatic Endothel

In [48]:
result[('VH_M_Skin', 'VH_M_skin')]

{'CD68 \n/ Macrophage': 140151,
 'T-Killer': 51356,
 'T-reg': 183849,
 'T-helper': 1313605}

In [49]:
file_cell_type_count_per_AS = './csv/cell_type_count_per_AS.csv'

with open(file_cell_type_count_per_AS, 'w', encoding='UTF-8', newline='') as f:
    csv_writer = csv.writer(f)
    for (organ, AS) in result:
        dic_cell_type_count = result[(organ, AS)]

        for cell_type, count in dic_cell_type_count.items():
            csv_writer.writerow([organ, AS, cell_type, count])

file_cell_type_count_per_AS_Skin = './csv/cell_type_count_per_AS_Skin.csv'
with open(file_cell_type_count_per_AS_Skin, 'w', encoding='UTF-8', newline='') as f:
    csv_writer = csv.writer(f)
    for (organ, AS) in result:
        if 'Skin' in organ:
            dic_cell_type_count = result[(organ, AS)]

            for cell_type, count in dic_cell_type_count.items():
                csv_writer.writerow([organ, AS, cell_type, count])

Create a table: Tissue block by HuBMAP IDs | AS it collides with (1) | AS it collides with (1) |...| AS it collides with (n)

In [6]:
file_mapping_rui_uuid_hubmap_id = './csv/mapping_rui_uuid_hubmap_id.csv'
file_hubmap_id_collision_AS = "./csv/hubmap_id_collision_AS.csv"

with open(file_mapping_rui_uuid_hubmap_id, 'r', encoding='UTF8') as f, open(file_hubmap_id_collision_AS, 'w', encoding='UTF-8', newline='') as f2:
    csv_reader = csv.reader(f)
    csv_writer = csv.writer(f2)
    csv_writer.writerow(['HubMAP ID', 'ASs'])
    i = 0

    for row in csv_reader:

        if i == 0:
            i = 1
            continue

        hubmap_id, uuid, rui_url = row
        print(hubmap_id)
        get_response = requests.get(rui_url).json()
        rui_location = get_response['rui_location']
        post_response = requests.post(url=url_collision_detection, json=rui_location)

        try:
            post_response = post_response.json()
            collision_AS = []
            for AS_item in post_response:
                # percentage_of_AS = AS_item['percentage_of_AS']
                # organ = AS_item['organ']
                node_name = AS_item['node_name']
                collision_AS.append(node_name)
            csv_writer.writerow([hubmap_id] + collision_AS)
        except:
            print(post_response.content)


HBM876.HTRX.668
HBM537.GVTL.683
HBM447.KCHB.778
HBM552.PBBF.363
HBM282.BKRZ.254
HBM834.THPC.928
HBM958.GZCX.985
HBM684.TLPQ.738
HBM476.VHDB.268
HBM393.DZDZ.972
HBM942.NXWW.245
HBM427.KFXP.323
HBM634.QQWF.789
HBM664.LQSH.693
HBM873.NSHG.559
HBM258.HQCL.349
HBM243.MTZV.726
HBM252.JLSW.848
HBM246.KBZS.276
HBM495.ZFDT.229
HBM242.HNLH.466
HBM844.KXXX.692
HBM979.KNQM.454
HBM525.LRPL.332
HBM332.KQWB.454
HBM688.FLJB.475
HBM347.MRTV.333
HBM996.XSNK.694
HBM669.GWVL.893
HBM399.TFTQ.282
HBM382.HNKL.447
HBM537.LTFK.379
HBM969.VNQL.553
HBM892.MXFF.848
HBM992.XNZH.647
HBM574.NFCS.842
HBM224.GLDC.549
HBM264.RWRW.668
HBM773.WCXC.264
HBM268.RGJT.587
HBM432.BPZM.698
HBM578.SRZG.769
HBM253.HFDZ.866
HBM894.PGJZ.258
HBM229.HKHH.537
HBM865.PBJW.845
HBM396.WPKL.954
HBM839.TQNM.958
HBM764.ZCQR.585
HBM797.PDPS.368
HBM982.TPNQ.737
HBM375.ZKZZ.765
HBM654.BKGL.942
HBM639.SGGN.469
HBM297.VBLS.984


In [26]:
df_specification = pd.read_csv('csv/table5big-test.csv',  header=None)

In [27]:
df_specification

,0,1,2,3
0,VH_M_Skin,VH_M_skin,CD68 \n/ Macrophage,62671
1,VH_M_Skin,VH_M_skin,T-Killer,14891
2,VH_M_Skin,VH_M_skin,T-reg,49951
3,VH_M_Skin,VH_M_skin,T-helper,532775
4,VH_M_Skin,VH_M_skin,CD68,4035
...,...,...,...,...
508,VH_M_Heart,VH_M_right_cardiac_atrium,Fibroblast,97210
509,VH_M_Heart,VH_M_right_cardiac_atrium,Glia,3291
510,VH_M_Heart,VH_M_right_cardiac_atrium,Immune (lymphocyte),10531
511,VH_M_Heart,VH_M_right_cardiac_atrium,Immune (myeloid),119832


In [28]:
df_VH_F_Kidney_L = df_specification[df_specification.iloc[:, 0] == 'VH_F_Kidney_L']

In [29]:
df_VH_F_Kidney_L

,0,1,2,3
148,VH_F_Kidney_L,VH_F_renal_pyramid_L_a,Afferent / Efferent Arteriole Endothelial,1273
149,VH_F_Kidney_L,VH_F_renal_pyramid_L_a,Ascending Thin Limb,6621
150,VH_F_Kidney_L,VH_F_renal_pyramid_L_a,Ascending Vasa Recta Endothelial,15025
151,VH_F_Kidney_L,VH_F_renal_pyramid_L_a,Classical Dendritic,254
152,VH_F_Kidney_L,VH_F_renal_pyramid_L_a,Connecting Tubule,213669
...,...,...,...,...
350,VH_F_Kidney_L,VH_F_renal_pyramid_L_h,Vascular Smooth Muscle / Pericyte,1656
351,VH_F_Kidney_L,VH_F_renal_pyramid_L_h,Descending Vasa Recta Endothelial,940
352,VH_F_Kidney_L,VH_F_renal_pyramid_L_h,Distal Convoluted Tubule Type 1,5105
353,VH_F_Kidney_L,VH_F_renal_pyramid_L_h,M2 Macrophage,1388


In [31]:
df_VH_F_Kidney_L.to_csv('csv/specification_VH_F_Kidney_L.csv', index=False, header=None)